In [7]:

lyrics = """
(Intro)
Hey, it's KAYPramitO, light the room
We're headed to Florida, in a Zoom

(Verse 1)
Pramit said, "Where do you work? Oh, Wall Street Company!"
Everyone's talking, so much synergy
Neon lights and city nights
Deadlock season's the latest hype

(Chorus)
Let's go shower, flash and tower
30 minutes to power, this is our hour
Grab the orb, shower scene
Cypher’s here, oh how we've been

(Verse 2)
Sova, Reyna, Phoenix too
Headphones on backwards, what a view
Jon says, "Try not to die"
Lucy dings Phoenix, Naiyo in the sky

(Chorus)
Let's go shower, flash and tower
30 minutes to power, this is our hour
Pramit with the AWP, nothing sour
Holding angles, gaming flower

(Bridge)
What’s that dog doing? Jimmy wants to know
Reyna’s lurking, sneaky like a Poe
Get in position, all set, let's roll
Round and round, capture the soul

(Outro)
KAYPramitO, take us out
In this game, we’re never in doubt
Shower's clear, shine bright, no fear
We play for fun, year after year
"""


In [43]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip
from moviepy.video.fx.loop import loop
import ffmpeg
import asyncio
import time
import os

async def merge_videos_and_song_with_subtitles(srt_path):
    # Step 1: Download song and videos
    # async with ClientSession() as session:
    #     song_path = await download_file(session, song_url, 'song.mp3')
    #     video_paths = []
    #     for i, video_url in enumerate(video_urls):
    #         video_path = await download_file(session, video_url, f'video_{i+1}.mp4')
    #         video_paths.append(video_path)

    song_path = './song.mp3'
    video_paths = [f'./video_{i+1}.mp4' for i in range(6)]

    # Step 2: Load videos using MoviePy
    video_clips = [VideoFileClip(video) for video in video_paths]
    
    # Step 3: Concatenate and loop the videos until they match the song duration
    concatenated_clip = concatenate_videoclips(video_clips)

    song_audio = AudioFileClip(song_path)
    song_duration = song_audio.duration
    final_video_clip = loop(concatenated_clip, duration=song_duration)
    final_video_clip = final_video_clip.set_audio(song_audio)

    # Step 4: Add subtitles using ffmpeg (MoviePy)
    # MoviePy uses ffmpeg to overlay the subtitles without ImageMagick.
    final_video_with_subtitles = final_video_clip.subclip(0, song_duration).set_duration(song_duration).set_audio(song_audio)

    # Step 5: Write the output to a file
    output_path = f"final_output_with_subtitles_{time.time()}.mp4"
    final_video_with_subtitles.write_videofile(output_path, codec='libx265', audio_codec='aac', ffmpeg_params=['-vf', f"subtitles={srt_path}", '-crf', '28'])

    # Step 6: Clean up temporary files
    song_audio.close()
    final_video_clip.close()
    concatenated_clip.close()
    final_video_with_subtitles.close()
    for video in video_clips:
        video.close()
    
    return output_path

In [44]:
await merge_videos_and_song_with_subtitles('./output.ass')


Moviepy - Building video final_output_with_subtitles_1729416223.7528567.mp4.
MoviePy - Writing audio in final_output_with_subtitles_1729416223.7528567TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_output_with_subtitles_1729416223.7528567.mp4



Moviepy - Done !
Moviepy - video ready final_output_with_subtitles_1729416223.7528567.mp4


'final_output_with_subtitles_1729416223.7528567.mp4'

In [4]:
import stable_whisper

In [5]:
model = stable_whisper.load_model("base")

In [8]:
# remove everything in brackets of any sory from lyrics
import re
lyrics = re.sub(r'\[.*?\]|\(.*?\)', '', lyrics)


In [37]:
result = model.align('./song.mp3', lyrics, language='en', fast_mode=True, regroup=False)


Align: 100%|██████████| 197.42/197.42 [00:00<00:00, 248.88sec/s]
Adjustment: 100%|██████████| 83.78/83.78 [00:00<00:00, 33514.43sec/s]


In [39]:
result

In [40]:
for segment in result:
    print("[%.2f -> %.2f] %s" % (segment.start, segment.end, segment.text))



[0.00 -> 83.78]   Hey, it's KAYPramitO, light the room We're headed to Florida, in a Zoom   Pramit said, "Where do you work? Oh, Wall Street Company!" Everyone's talking, so much synergy Neon lights and city nights Deadlock season's the latest hype   Let's go shower, flash and tower 30 minutes to power, this is our hour Grab the orb, shower scene Cypher’s here, oh how we've been   Sova, Reyna, Phoenix too Headphones on backwards, what a view Jon says, "Try not to die" Lucy dings Phoenix, Naiyo in the sky   Let's go shower, flash and tower 30 minutes to power, this is our hour Pramit with the AWP, nothing sour Holding angles, gaming flower   What’s that dog doing? Jimmy wants to know Reyna’s lurking, sneaky like a Poe Get in position, all set, let's roll Round and round, capture the soul   KAYPramitO, take us out In this game, we’re never in doubt Shower's clear, shine bright, no fear We play for fun, year after year 


In [41]:
srt = result.to_ass('output.ass')


Saved: c:\Users\pmazu\Documents\GitHub\BlinkStory\output.ass


In [6]:
import os
import google.auth
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.http import MediaFileUpload
from google.oauth2.credentials import Credentials

class YouTubeUploader:
    def __init__(self, client_secret_file, api_service_name="youtube", api_version="v3", scopes=None):
        if scopes is None:
            scopes = ["https://www.googleapis.com/auth/youtube.upload"]
        
        self.client_secret_file = client_secret_file
        self.api_service_name = api_service_name
        self.api_version = api_version
        self.scopes = scopes
        self.credentials = None
        self.youtube = None
        self.token_file = 'token.json'

    def authenticate(self):
        # OAuth 2.0 flow to get authenticated credentials
        if os.path.exists('token.json'):
            self.credentials = Credentials.from_authorized_user_file('token.json', self.scopes)
                
        # If no valid credentials, perform the OAuth flow
        if not self.credentials or not self.credentials.valid:
            if self.credentials and self.credentials.expired and self.credentials.refresh_token:
                # Refresh the credentials if they're expired
                self.credentials.refresh(google.auth.transport.requests.Request())
            else:
                # OAuth 2.0 flow to get authenticated credentials for the first time
                flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
                    self.client_secret_file, self.scopes
                )
                self.credentials = flow.run_local_server()

            # Save the credentials to a file for the next time
            with open(self.token_file, 'w') as token:
                token.write(self.credentials.to_json())
        
        # Build the YouTube service object
        self.youtube = googleapiclient.discovery.build(
            self.api_service_name, self.api_version, credentials=self.credentials
        )

    def upload_video(self, video_path, title, description, tags=None, category_id="22", privacy_status="private"):
        if tags is None:
            tags = []

        # Set up the video upload details
        body = {
            "snippet": {
                "title": title,
                "description": description,
                "tags": tags,
                "categoryId": category_id  # Category: 22 is for 'People & Blogs'
            },
            "status": {
                "privacyStatus": privacy_status  # "public", "private", or "unlisted"
            }
        }

        # Upload the video file using the YouTube Data API
        media = MediaFileUpload(video_path, chunksize=-1, resumable=True)

        request = self.youtube.videos().insert(
            part="snippet,status",
            body=body,
            media_body=media
        )

        response = None
        while response is None:
            status, response = request.next_chunk()
            if status:
                print(f"Uploaded {int(status.progress() * 100)}%")

        print("Upload complete!")
        print(f"Video ID: {response['id']}")
        return response

In [8]:

uploader = YouTubeUploader("client_secret.json")
uploader.authenticate()

In [9]:
uploader.upload_video(
    video_path="./final_output.mp4",
    title="Your Video Title",
    description="Description of your video",
    tags=["sample", "video"],
    category_id="22",  # Category ID for 'People & Blogs'
    privacy_status="private"
)

Upload complete!
Video ID: okXnGcHslyc


{'kind': 'youtube#video',
 'etag': 'm3oklTOzemlDpsolC95uDzEHTDI',
 'id': 'okXnGcHslyc',
 'snippet': {'publishedAt': '2024-10-20T09:46:43Z',
  'channelId': 'UCLVW6uqN6az8ReX015UkpCA',
  'title': 'Your Video Title',
  'description': 'Description of your video',
  'thumbnails': {'default': {'url': 'https://i9.ytimg.com/vi/okXnGcHslyc/default.jpg?sqp=CJyi07gG&rs=AOn4CLAh9GoWOYBWNLPnWdmUH_-x31uiVA',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i9.ytimg.com/vi/okXnGcHslyc/mqdefault.jpg?sqp=CJyi07gG&rs=AOn4CLDcOV__AKaVnESXLYpJPdG5nKxmWw',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i9.ytimg.com/vi/okXnGcHslyc/hqdefault.jpg?sqp=CJyi07gG&rs=AOn4CLBfSjzKjP5dsKMbR470bdeNXmpRsw',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Pramit Mazumder',
  'tags': ['sample', 'video'],
  'categoryId': '22',
  'liveBroadcastContent': 'none',
  'localized': {'title': 'Your Video Title',
   'description': 'Description of your video'}},
 'status': {'uploadSta